In [16]:
#import libraries

import re
import nltk
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report


In [17]:
# Change display setting to show full text
pd.set_option('display.max_colwidth', None)

tweet = pd.read_csv("tweets.csv")

In [18]:
tweet.head()

,Unnamed: 0,tweet,sentiment
0,0,"You just know there are 8 or 9 ""Climate NGOs"" with Africa offices in Nairobi currently falling over themselves to offer $1,000 grants to some stupid Nigerian journalists to report the flooding in Maiduguri as ""climate change,"" tying in an argument against Africa using its own fossil fuels to industrialise.\n\nFor the record, here are the facts: this disaster was caused by the collapse of Maiduguri's 40 year-old Alau Dam, a dam constructed to shield Maiduguri from seasonal flooding along the Ngadda river basin. This dam already suffered structural damage due to high seasonal rainfall in 1992, 1994 and 2012, and never received the required maintenance to put it back into ideal operational shape.\n\nIn other words, this is an infrastructure problem, not a ""climate"" problem. It is a problem that will be solved by developing more capacity to build and adequately maintain infrastructure like Europe does - capacity that comes with cheap energy and industrialisation like Europe did.\n\nIf you are the oloriburuku bastard journalist in Abuja that is preparing to submit that $1,000 grant application to write nonsense for white people and hurt your people in the process, you will contract untreatable genital herpes and suffer until you die.",0
1,1,"THE GREAT AWAKENING: The Truth about Bill Gates, Klaus Schwab, the World Economic Forum, BlackRock and Governments around the world destroying the Family unit and Sovereign Nations for a one world government through deliberate chaos, division, wars, climate change and pandemics.\n\nWe have the right as citizens in this country to abolish government and create a new one and that is the authority that has been given to us. It is written in our documents, the Declaration of Independence, the Constitution of the United States, the Declaration of Human Rights. We have indelible rights and the basis of those rights are our freedoms and if we don't have that, we don't have life. Arise humanity, wake up to the truth, fight for your freedom and for your lives.",0
2,2,"Heavy rainfall in Ireland is becoming around 20% more intense and severe, due to human-induced climate change.\nCouncil has recommended prioritisation of projects that enhance the resilience of Ireland’s critical infrastructure.\n📑Read more: https://t.co/Mlbgp0qB2K https://t.co/4lg6vnW7D6",1
3,3,Why is Ginni Thomas still free ?\nWhy isn’t Mark Meadows on trial ?\nWhy isn’t Clarence Thomas in prison ?\nWhy isn’t Trump facing execution for Treason ?\n\nThese are the most pressing issues in the USA ..other than Climate Change..which will absolutely end our way of life.\n\nWTAF https://t.co/3YT5DY0msl,0
4,4,"Jasper, Alberta, one of the most beautiful places in the world. Firefighters don't think anything will be left by morning. \n\nI can't stop crying.\n\nDanielle Smith says there's no climate change, and no need to allocate money for forest fires.\n\nThe Conservatives destroyed Jasper. https://t.co/qgxilRMyPJ",0


In [19]:
#Rename the column

tweet1 = tweet.rename(columns={'Unnamed: 0': 'Id'}, inplace=False)
tweet1.head()

,Id,tweet,sentiment
0,0,"You just know there are 8 or 9 ""Climate NGOs"" with Africa offices in Nairobi currently falling over themselves to offer $1,000 grants to some stupid Nigerian journalists to report the flooding in Maiduguri as ""climate change,"" tying in an argument against Africa using its own fossil fuels to industrialise.\n\nFor the record, here are the facts: this disaster was caused by the collapse of Maiduguri's 40 year-old Alau Dam, a dam constructed to shield Maiduguri from seasonal flooding along the Ngadda river basin. This dam already suffered structural damage due to high seasonal rainfall in 1992, 1994 and 2012, and never received the required maintenance to put it back into ideal operational shape.\n\nIn other words, this is an infrastructure problem, not a ""climate"" problem. It is a problem that will be solved by developing more capacity to build and adequately maintain infrastructure like Europe does - capacity that comes with cheap energy and industrialisation like Europe did.\n\nIf you are the oloriburuku bastard journalist in Abuja that is preparing to submit that $1,000 grant application to write nonsense for white people and hurt your people in the process, you will contract untreatable genital herpes and suffer until you die.",0
1,1,"THE GREAT AWAKENING: The Truth about Bill Gates, Klaus Schwab, the World Economic Forum, BlackRock and Governments around the world destroying the Family unit and Sovereign Nations for a one world government through deliberate chaos, division, wars, climate change and pandemics.\n\nWe have the right as citizens in this country to abolish government and create a new one and that is the authority that has been given to us. It is written in our documents, the Declaration of Independence, the Constitution of the United States, the Declaration of Human Rights. We have indelible rights and the basis of those rights are our freedoms and if we don't have that, we don't have life. Arise humanity, wake up to the truth, fight for your freedom and for your lives.",0
2,2,"Heavy rainfall in Ireland is becoming around 20% more intense and severe, due to human-induced climate change.\nCouncil has recommended prioritisation of projects that enhance the resilience of Ireland’s critical infrastructure.\n📑Read more: https://t.co/Mlbgp0qB2K https://t.co/4lg6vnW7D6",1
3,3,Why is Ginni Thomas still free ?\nWhy isn’t Mark Meadows on trial ?\nWhy isn’t Clarence Thomas in prison ?\nWhy isn’t Trump facing execution for Treason ?\n\nThese are the most pressing issues in the USA ..other than Climate Change..which will absolutely end our way of life.\n\nWTAF https://t.co/3YT5DY0msl,0
4,4,"Jasper, Alberta, one of the most beautiful places in the world. Firefighters don't think anything will be left by morning. \n\nI can't stop crying.\n\nDanielle Smith says there's no climate change, and no need to allocate money for forest fires.\n\nThe Conservatives destroyed Jasper. https://t.co/qgxilRMyPJ",0


In [20]:
tweet1.tail()

,Id,tweet,sentiment
996,996,"Kamala hires ‘climate’ aide who believes climate change is reason to not have children, promoted gas stove bans https://t.co/JbuxkKVJH8",0
997,997,This climate change fix could save the world — or doom it - USA TODAY https://t.co/YnoMQwzPBe,1
998,998,"This is howling into the void, but if they had any integrity the climate sceptics who this week used data on the UK's slightly cooler summer to suggest climate change is no big deal would look at this new global data and experience crushing embarrassment. https://t.co/QKqGcN7Ap2",0
999,999,This dude really said that “Climate change is having a disproportionate effect on the physical and mental health of black communities”\n\nCmon 🤣🤣\nhttps://t.co/SLLhjiRElG,0
1000,1000,"Hey Florida - bang up fucking job your governor is doing. If the governor of the likely first state to become submerged didn't run around stating cLiMaTe cHaNgE iS a HoAx perhaps measures could be taken to mitigate shit like this, no?",0


In [21]:
#Data cleaning - Remove hastags,urls, emojis and special characters
#Define function to clean tweets


def clean_tweet(tweet1):
    tweet1 = re.sub(r'http\S+', '', tweet1)  # Remove URLs
    tweet1 = re.sub(r'#\w+', '', tweet1)     # Remove hashtags
    tweet1 = re.sub(r'@[A-Za-z0-9_]+', '', tweet1)  # Remove mentions
    tweet1 = re.sub(r'[^A-Za-z\s]', '', tweet1)     # Remove special characters (keep only letters and spaces)
    tweet1 = re.sub(r'\s+', ' ', tweet1).strip()    # Remove extra spaces
    return tweet1

# Apply the clean_tweet function to the 'tweet' column
tweet1['cleaned_tweet'] = tweet1['tweet'].apply(clean_tweet)

In [22]:
tweet1.head()

,Id,tweet,sentiment,cleaned_tweet
0,0,"You just know there are 8 or 9 ""Climate NGOs"" with Africa offices in Nairobi currently falling over themselves to offer $1,000 grants to some stupid Nigerian journalists to report the flooding in Maiduguri as ""climate change,"" tying in an argument against Africa using its own fossil fuels to industrialise.\n\nFor the record, here are the facts: this disaster was caused by the collapse of Maiduguri's 40 year-old Alau Dam, a dam constructed to shield Maiduguri from seasonal flooding along the Ngadda river basin. This dam already suffered structural damage due to high seasonal rainfall in 1992, 1994 and 2012, and never received the required maintenance to put it back into ideal operational shape.\n\nIn other words, this is an infrastructure problem, not a ""climate"" problem. It is a problem that will be solved by developing more capacity to build and adequately maintain infrastructure like Europe does - capacity that comes with cheap energy and industrialisation like Europe did.\n\nIf you are the oloriburuku bastard journalist in Abuja that is preparing to submit that $1,000 grant application to write nonsense for white people and hurt your people in the process, you will contract untreatable genital herpes and suffer until you die.",0,You just know there are or Climate NGOs with Africa offices in Nairobi currently falling over themselves to offer grants to some stupid Nigerian journalists to report the flooding in Maiduguri as climate change tying in an argument against Africa using its own fossil fuels to industrialise For the record here are the facts this disaster was caused by the collapse of Maiduguris yearold Alau Dam a dam constructed to shield Maiduguri from seasonal flooding along the Ngadda river basin This dam already suffered structural damage due to high seasonal rainfall in and and never received the required maintenance to put it back into ideal operational shape In other words this is an infrastructure problem not a climate problem It is a problem that will be solved by developing more capacity to build and adequately maintain infrastructure like Europe does capacity that comes with cheap energy and industrialisation like Europe did If you are the oloriburuku bastard journalist in Abuja that is preparing to submit that grant application to write nonsense for white people and hurt your people in the process you will contract untreatable genital herpes and suffer until you die
1,1,"THE GREAT AWAKENING: The Truth about Bill Gates, Klaus Schwab, the World Economic Forum, BlackRock and Governments around the world destroying the Family unit and Sovereign Nations for a one world government through deliberate chaos, division, wars, climate change and pandemics.\n\nWe have the right as citizens in this country to abolish government and create a new one and that is the authority that has been given to us. It is written in our documents, the Declaration of Independence, the Constitution of the United States, the Declaration of Human Rights. We have indelible rights and the basis of those rights are our freedoms and if we don't have that, we don't have life. Arise humanity, wake up to the truth, fight for your freedom and for your lives.",0,THE GREAT AWAKENING The Truth about Bill Gates Klaus Schwab the World Economic Forum BlackRock and Governments around the world destroying the Family unit and Sovereign Nations for a one world government through deliberate chaos division wars climate change and pandemics We have the right as citizens in this country to abolish government and create a new one and that is the authority that has been given to us It is written in our documents the Declaration of Independence the Constitution of the United States the Declaration of Human Rights We have indelible rights and the basis of those rights are our freedoms and if we dont have that we dont have life Arise humanity wake up to the truth fight for your freedom and for your lives
2,2,"Heavy rain

In [23]:
tweet1.tail()

,Id,tweet,sentiment,cleaned_tweet
996,996,"Kamala hires ‘climate’ aide who believes climate change is reason to not have children, promoted gas stove bans https://t.co/JbuxkKVJH8",0,Kamala hires climate aide who believes climate change is reason to not have children promoted gas stove bans
997,997,This climate change fix could save the world — or doom it - USA TODAY https://t.co/YnoMQwzPBe,1,This climate change fix could save the world or doom it USA TODAY
998,998,"This is howling into the void, but if they had any integrity the climate sceptics who this week used data on the UK's slightly cooler summer to suggest climate change is no big deal would look at this new global data and experience crushing embarrassment. https://t.co/QKqGcN7Ap2",0,This is howling into the void but if they had any integrity the climate sceptics who this week used data on the UKs slightly cooler summer to suggest climate change is no big deal would look at this new global data and experience crushing embarrassment
999,999,This dude really said that “Climate change is having a disproportionate effect on the physical and mental health of black communities”\n\nCmon 🤣🤣\nhttps://t.co/SLLhjiRElG,0,This dude really said that Climate change is having a disproportionate effect on the physical and mental health of black communities Cmon
1000,1000,"Hey Florida - bang up fucking job your governor is doing. If the governor of the likely first state to become submerged didn't run around stating cLiMaTe cHaNgE iS a HoAx perhaps measures could be taken to mitigate shit like this, no?",0,Hey Florida bang up fucking job your governor is doing If the governor of the likely first state to become submerged didnt run around stating cLiMaTe cHaNgE iS a HoAx perhaps measures could be taken to mitigate shit like this no


In [24]:
tweet1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             1001 non-null   int64 
 1   tweet          1001 non-null   object
 2   sentiment      1001 non-null   int64 
 3   cleaned_tweet  1001 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB


In [25]:
tweet1.isnull().sum()

Id               0
tweet            0
sentiment        0
cleaned_tweet    0
dtype: int64

In [26]:
#Keyword-Specific Feature Extraction

tweet1['keyword_present'] = tweet1['cleaned_tweet'].str.contains("climate change")
tweet1['has_hashtags'] = tweet1['tweet'].apply(lambda x: re.findall(r"#(\w+)", x))
print(tweet1[['cleaned_tweet', 'keyword_present', 'has_hashtags']])

In [27]:
#Using NLP for Contextual Understanding

nltk.download('vader_lexicon')

# Sentiment Analysis
sia = SentimentIntensityAnalyzer()

tweet1['sentiment'] = tweet1['cleaned_tweet'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Label sentiment: Positive (1), Neutral (0), Negative (-1)
tweet1['sentiment_label'] = tweet1['sentiment'].apply(lambda x: 1 if x < 0 else (-1 if x < 0 else 0))

# Display the first few rows with sentiment analysis
print(tweet1.head())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


   Id  \
0   0   
1   1   
2   2   
3   3   
4   4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [28]:
#Train model
# Feature Extraction with TF-IDF

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(tweet1['cleaned_tweet']).toarray()

# Labels (For illustration, assume 0 = Not disinformation, 1 = Disinformation)
# You will need labeled data for this step
y = tweet1['keyword_present']  # Placeholder label

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model training
model = LogisticRegression()
model.fit(X_train, y_train)

# Model evaluation
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.6976744186046512
              precision    recall  f1-score   support

       False       1.00      0.07      0.13        98
        True       0.69      1.00      0.82       203

    accuracy                           0.70       301
   macro avg       0.85      0.54      0.48       301
weighted avg       0.79      0.70      0.59       301



In [29]:
import joblib

# Save the TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [30]:
# Save the Logistic Regression model
joblib.dump(model, 'logistic_model.pkl')

['logistic_model.pkl']

In [ ]:
#Integration of model with Twitter Scraper
#Real-Time or Batch Data Processing

